In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from helpers import Opt_Helpers, BT_Helpers
from pandas_datareader import data

In [3]:
"""Define parameters for data request."""

tickers = ['AGG', 'EBND', 'URTH', 'EEM']
data_source = 'yahoo'
start_date = '2014-03-31'
end_date = '2018-03-31'

In [4]:
"""Request data, ensure sorted by ascending date, and normalise."""

panel_data = data.DataReader(tickers, data_source, start_date, end_date)
px_data = panel_data.loc['Adj Close'].dropna().sort_index(ascending=True)
px_data = px_data / px_data.iloc[0]

In [5]:
"""Initialise optimisation class with price data."""

opt = Opt_Helpers(px_data, rf=0, scaling_fact=252)

In [6]:
"""
TEST CASE
Add constraints: Portfolio weights must sum to 100%, minimum ann. return > 3.0%,
Add boundaries: Long only, no short positions.
"""

min_ret = 0.035

cons = (
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
    {'type': 'ineq', 'fun': lambda w: opt.summary_stats(w)['exp_return'] - min_ret}
)

bnds = tuple((0, 1) for x in px_data.columns)

In [7]:
"""Run optimisation."""

opt.port_optimisation(opt.max_dr, constraints=cons, bounds=bnds)

Optimization terminated successfully. Success==True after 7 iterations.
Total time: 0.20122885704040527 secs


In [8]:
"""Class attributes store optimised weights."""

opt.weights

array([  7.38950123e-01,   6.81152850e-04,   3.26951070e-02,
         2.27673617e-01])

In [9]:
"""Initialise backtest class."""

bt = BT_Helpers(px_data, opt_period = 365, val_period = 90, rf=0, scaling_fact=252)

In [10]:
"""Generate historical optimisation weights."""

bt_res = bt.bt_optimisation(opt.max_dr, constraints=cons, bounds=bnds)

Optimization terminated successfully. Success==True after 6 iterations.
Total time: 0.16022658348083496 secs
Optimization terminated successfully. Success==True after 6 iterations.
Total time: 0.17048883438110352 secs
Optimization terminated successfully. Success==True after 6 iterations.
Total time: 0.1844329833984375 secs
Optimization terminated successfully. Success==True after 5 iterations.
Total time: 0.1406397819519043 secs
Optimization terminated successfully. Success==True after 5 iterations.
Total time: 0.14734792709350586 secs
Optimization terminated successfully. Success==True after 3 iterations.
Total time: 0.07813286781311035 secs
Optimization terminated successfully. Success==True after 3 iterations.
Total time: 0.07813358306884766 secs
Optimization terminated successfully. Success==True after 3 iterations.
Total time: 0.09603643417358398 secs
Optimization terminated successfully. Success==True after 5 iterations.
Total time: 0.14933276176452637 secs
Optimization terminat